In [1]:
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

In [2]:
data_csv = pd.read_csv(r"C:\Users\mahen\MLOPS2\data\heart_2020_cleaned.csv")

In [3]:
columns = data_csv.columns.values
rand = random.randint(0, len(data_csv) - 1)

# Mengambil fitur
features = data_csv.values[rand]

# Membuat dictionary input
inputs = {key: value for key, value in zip(columns, features)}
inputs

{'HeartDisease': 0,
 'HeartDiseaseStatus': 'No',
 'BMI': 26.63,
 'Smoking': 'Yes',
 'AlcoholDrinking': 'No',
 'Stroke': 'No',
 'PhysicalHealth': 30,
 'MentalHealth': 0,
 'DiffWalking': 'Yes',
 'Sex': 'Male',
 'AgeCategory': '60-64',
 'Race': 'American Indian/Alaskan Native',
 'Diabetic': 'No',
 'PhysicalActivity': 'No',
 'GenHealth': 'Poor',
 'SleepTime': 8,
 'Asthma': 'No',
 'KidneyDisease': 'No',
 'SkinCancer': 'No'}

In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [8]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://mahendradwikm-pipeline-production.up.railway.app/v1/models/heart-disease-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.5:
        return "Tidak Mengalami Penyakit Jantung"
    else:
        return "Mengalami Penyakit Jantung"

In [9]:
hasil_prediksi = make_predictions(inputs)
print(f'Hasil Prediksi: {hasil_prediksi}')
print('Fitur Input:', inputs)

Hasil Prediksi: Tidak Mengalami Penyakit Jantung
Fitur Input: {'HeartDisease': 0, 'HeartDiseaseStatus': 'No', 'BMI': 26.63, 'Smoking': 'Yes', 'AlcoholDrinking': 'No', 'Stroke': 'No', 'PhysicalHealth': 30, 'MentalHealth': 0, 'DiffWalking': 'Yes', 'Sex': 'Male', 'AgeCategory': '60-64', 'Race': 'American Indian/Alaskan Native', 'Diabetic': 'No', 'PhysicalActivity': 'No', 'GenHealth': 'Poor', 'SleepTime': 8, 'Asthma': 'No', 'KidneyDisease': 'No', 'SkinCancer': 'No'}
